In [1]:
""" Repeating MNIST first experiment of swish paper. """

import gc # Garbage collector
import logging
import numpy as np
import matplotlib.pyplot as plt

# Record settings
LOG_FORMAT = "%(levelname)s %(asctime)s - %(message)s"
logging.basicConfig(filename="swish_first_exp_log.txt",format = LOG_FORMAT, level = logging.DEBUG, filemode = "a")
logs = logging.getLogger()

%matplotlib notebook

np.random.seed(2)

from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# For adding new activation function
from keras import backend as K
from keras.datasets import mnist
from keras.utils.generic_utils import get_custom_objects
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print("X_train original shape", X_train.shape)
print("y_train original shape", Y_train.shape)

X_train original shape (60000, 28, 28)
y_train original shape (60000,)


In [3]:
# Normalization
nb_classes = 10
# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

In [4]:
# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

In [5]:
# Set the random seed
random_seed = 2

In [6]:
# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)

In [7]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.reshape(-1,784)
X_val = X_val.reshape(-1,784)
X_test = X_test.reshape(-1,784)
# test = test.values.reshape(-1,28,28,1)
print(X_train.shape, X_val.shape, X_test.shape)

(54000, 784) (6000, 784) (10000, 784)


In [8]:
def swish(x):
    return x*K.sigmoid(x)

def e_swish_2(x):
    sigmoid = K.sigmoid(x)
    return K.maximum(x*sigmoid, x*(2-sigmoid))

In [9]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
def create(act, n):
    model = Sequential()
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation(act))
    for i in range(n-1):
        if i%2 == 0:
            model.add(BatchNormalization())
        model.add(Dense(512))
        model.add(Activation(act))
        model.add(Dropout(0.3))
        
    model.add(Dense(nb_classes))
    model.add(Activation("softmax"))
    
    return model

In [10]:
def plot(arr, names = ["relu", "e_swish_2", "swish"]):
    fig, ax = plt.subplots()
    for item in arr:
        ax.plot(np.array(item))
    
    plt.grid()
    plt.legend(names[:len(arr)], loc='upper right')
    ax.set_ylim(0,1)
    plt.show()

In [11]:
def accuracy(y_pred, y_test):
    y_hat = np.argmax(y_pred, axis=1)
    y = np.argmax(y_test, axis=1)

    good = np.sum(np.equal(y, y_hat))
    return float(good/len(y_test))

In [ ]:
act = "relu"

logs_relu = []
record_relu = []
for n in range(23,42,3):
    ensembler = 0
    logger = [n]
    print("\n \n Starting round with {0} layers".format(n))
    for i in range(3):
        # Garbage collector
        gc.collect()
        # Set optimizer
        opt = SGD(lr=0.01, momentum=0.9)
        # Set callbacks (learning rate reducer and early stopping)
        learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=2, verbose=1, factor=0.35, min_lr=0.00001)
        early_stop = EarlyStopping(monitor='val_acc', patience=5, verbose = 1)
        # Common params 
        epochs = 15
        batch_size = 128
        # Create and compile the model
        model = create(act, n)
        # Compile the model
        model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
        # Train the model
        history = model.fit(X_train,Y_train, epochs = epochs, validation_data = (X_val,Y_val),
                            verbose = 1 , callbacks=[learning_rate_reduction, early_stop])
        
        # Record accuracy of each model and save it
        logger.append(model.evaluate(X_test, Y_test)[1])
        # Calculate probabilities of test data and sum them toghether
        ensembler += model.predict_proba(X_test)
        # Clear session (GPU MEMORY)
        K.get_session().close()
        K.set_session(K.tf.Session())
        del model, history, learning_rate_reduction, early_stop, opt
     
    # Calculate the median accuracy
    ensembled = accuracy(ensembler, Y_test)
    print(ensembled)
    # Save the ensembled accuracy and the three models accuracy
    record_relu.append([n, ensembled])
    logs_relu.append(logger)
    del ensembler, ensembled
    
print("\n \n \n")
print("Logs Relu: ", logs_relu)
print("Record Relu: ", record_relu)

    
plot([[x[1] for x in logs_relu]])


 
 Starting round with 23 layers
Train on 54000 samples, validate on 6000 samples
Epoch 1/15
54000/54000 [==============================] - 74s 1ms/step - loss: 1.8138 - acc: 0.3087 - val_loss: 1.1260 - val_acc: 0.5507
Epoch 2/15
54000/54000 [==============================] - 70s 1ms/step - loss: 1.2893 - acc: 0.5202 - val_loss: 0.9207 - val_acc: 0.6202
Epoch 3/15
54000/54000 [==============================] - 70s 1ms/step - loss: 1.0644 - acc: 0.6229 - val_loss: 0.5850 - val_acc: 0.7962
Epoch 4/15
54000/54000 [==============================] - 70s 1ms/step - loss: 0.8988 - acc: 0.7164 - val_loss: 0.4861 - val_acc: 0.8677
Epoch 5/15
54000/54000 [==============================] - 70s 1ms/step - loss: 0.7803 - acc: 0.7758 - val_loss: 0.3949 - val_acc: 0.8998
Epoch 6/15
54000/54000 [==============================] - 70s 1ms/step - loss: 0.7047 - acc: 0.8082 - val_loss: 0.3282 - val_acc: 0.9175
Epoch 7/15
54000/54000 [==============================] - 70s 1ms/step - loss: 0.6350 - acc: 0.

54000/54000 [==============================] - 86s 2ms/step - loss: 1.2646 - acc: 0.5022 - val_loss: 1.0034 - val_acc: 0.5597
Epoch 9/15
54000/54000 [==============================] - 87s 2ms/step - loss: 1.2213 - acc: 0.5196 - val_loss: 0.9305 - val_acc: 0.5847
Epoch 10/15
54000/54000 [==============================] - 87s 2ms/step - loss: 1.1738 - acc: 0.5393 - val_loss: 0.9307 - val_acc: 0.6253: 1.1738 - a
Epoch 11/15
54000/54000 [==============================] - 87s 2ms/step - loss: 1.1366 - acc: 0.5599 - val_loss: 0.8411 - val_acc: 0.6435
Epoch 12/15
54000/54000 [==============================] - 87s 2ms/step - loss: 1.0811 - acc: 0.5784 - val_loss: 0.7895 - val_acc: 0.6858
Epoch 13/15
54000/54000 [==============================] - 86s 2ms/step - loss: 1.0426 - acc: 0.5961 - val_loss: 0.7910 - val_acc: 0.6593
Epoch 14/15
54000/54000 [==============================] - 87s 2ms/step - loss: 1.0739 - acc: 0.5937 - val_loss: 0.7843 - val_acc: 0.6813
Epoch 15/15
10000/10000 [==========

Epoch 4/15
54000/54000 [==============================] - 98s 2ms/step - loss: 1.9537 - acc: 0.2649 - val_loss: 1.7453 - val_acc: 0.3308
Epoch 5/15
54000/54000 [==============================] - 100s 2ms/step - loss: 1.8740 - acc: 0.2913 - val_loss: 1.6371 - val_acc: 0.3460
Epoch 6/15
54000/54000 [==============================] - 100s 2ms/step - loss: 1.7921 - acc: 0.3177 - val_loss: 1.5252 - val_acc: 0.3503
Epoch 7/15
54000/54000 [==============================] - 99s 2ms/step - loss: 1.7234 - acc: 0.3306 - val_loss: 1.4995 - val_acc: 0.4037
Epoch 8/15
54000/54000 [==============================] - 100s 2ms/step - loss: 1.6706 - acc: 0.3464 - val_loss: 1.4149 - val_acc: 0.4100
Epoch 9/15
54000/54000 [==============================] - 101s 2ms/step - loss: 1.6176 - acc: 0.3560 - val_loss: 1.3720 - val_acc: 0.4208
Epoch 10/15
54000/54000 [==============================] - 101s 2ms/step - loss: 1.5825 - acc: 0.3633 - val_loss: 1.3573 - val_acc: 0.4270
Epoch 11/15
54000/54000 [==========

In [ ]:
act = "relu"
logs_relu = []
record_relu = [[23, 0.9717], [26, 0.9253], [29, 0.8639], [32, 0.6807], [35, 0.5834]]
logs.info("Relu 38-42")

for n in range(38,42,3):
    ensembler = 0
    logger = [n]
    print("\n \n Starting round with {0} layers".format(n))
    logs.info("\n \n Starting round with {0} layers".format(n))
    for i in range(3):
        # Garbage collector
        gc.collect()
        # Set optimizer
        opt = SGD(lr=0.01, momentum=0.9)
        # Set callbacks (learning rate reducer and early stopping)
        learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=2, verbose=1, factor=0.35, min_lr=0.00001)
        early_stop = EarlyStopping(monitor='val_acc', patience=5, verbose = 1)
        # Common params 
        epochs = 15
        batch_size = 128
        # Create and compile the model
        model = create(act, n)
        # Compile the model
        model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
        # Train the model
        history = model.fit(X_train,Y_train, epochs = epochs, validation_data = (X_val,Y_val),
                            verbose = 1 , callbacks=[learning_rate_reduction, early_stop])
        
        # Record accuracy of each model and save it
        logger.append(model.evaluate(X_test, Y_test)[1])
        logs.info("Accuracy "+str(i)+": "+str(logger[-1]))
        # Calculate probabilities of test data and sum them toghether
        ensembler += model.predict_proba(X_test)
        # Clear session (GPU MEMORY)
        K.get_session().close()
        K.set_session(K.tf.Session())
        del model, history, learning_rate_reduction, early_stop, opt
     
    # Calculate the median accuracy
    ensembled = accuracy(ensembler, Y_test)
    print(ensembled)
    # Save the ensembled accuracy and the three models accuracy
    record_relu.append([n, ensembled])
    logs_relu.append(logger)
    del ensembler, ensembled
    logs.info("Ensembled accuracy: "+str(record_relu[-1]))
    logs.info("Logs: "+str(logs_relu[-1]))
    
logs.info("\n \n \n")
logs.info("Logs e_swish_2: "+str(logs_relu))
logs.info("Record e_swish_2: "+str(record_relu))
print("\n \n \n")
print("Logs Relu: ", logs_relu)
print("Record Relu: ", record_relu)

    
plot([[x[1] for x in logs_relu]])


 
 Starting round with 38 layers
Train on 54000 samples, validate on 6000 samples
Epoch 1/15
54000/54000 [==============================] - 121s 2ms/step - loss: 2.4120 - acc: 0.1019 - val_loss: 2.3258 - val_acc: 0.1003
Epoch 2/15
54000/54000 [==============================] - 114s 2ms/step - loss: 2.3133 - acc: 0.1030 - val_loss: 2.3261 - val_acc: 0.1017
Epoch 3/15
54000/54000 [==============================] - 113s 2ms/step - loss: 2.3052 - acc: 0.1075 - val_loss: 2.3299 - val_acc: 0.1698
Epoch 4/15
54000/54000 [==============================] - 113s 2ms/step - loss: 2.3037 - acc: 0.1086 - val_loss: 2.3284 - val_acc: 0.1152
Epoch 5/15
54000/54000 [==============================] - 113s 2ms/step - loss: 2.3031 - acc: 0.1089 - val_loss: 2.3024 - val_acc: 0.1152
Epoch 6/15
53984/54000 [============================>.] - ETA: 0s - loss: 2.3026 - acc: 0.1092
Epoch 00006: reducing learning rate to 0.0034999999217689036.
54000/54000 [==============================] - 112s 2ms/step - loss: 2

Epoch 5/15
54000/54000 [==============================] - 120s 2ms/step - loss: 2.3017 - acc: 0.1115 - val_loss: 2.3003 - val_acc: 0.1152
Epoch 6/15
53984/54000 [============================>.] - ETA: 0s - loss: 2.3018 - acc: 0.1110
Epoch 00006: reducing learning rate to 0.0012249999563209713.
54000/54000 [==============================] - 120s 2ms/step - loss: 2.3018 - acc: 0.1110 - val_loss: 2.3084 - val_acc: 0.1153
Epoch 00006: early stopping
10000/10000 [==============================] - 4s 448us/step
0.1572

 
 

Logs Relu:  [[23, 0.9717], [26, 0.9253], [29, 0.8639], [32, 0.6807], [35, 0.5834], [38, 0.1135, 0.1135, 0.1135], [41, 0.15679999999999999, 0.1135, 0.11360000000000001]]
Record Relu:  [[38, 0.1135], [41, 0.1572]]


<IPython.core.display.Javascript object>

In [ ]:
act = e_swish_2
logs.info("\n \n \n"+"\n \n \n")
logs.info("ESWISH 23-42")

logs_e_swish_2 = []
record_e_swish_2 = []
for n in range(23,42,3):
    ensembler = 0
    logger = [n]
    logs.info("\n \n Starting round with {0} layers".format(n))
    print("\n \n Starting round with {0} layers".format(n))
    for i in range(3):
        # Garbage collector
        gc.collect()
        # Set optimizer
        opt = SGD(lr=0.01, momentum=0.9)
        # Set callbacks (learning rate reducer and early stopping)
        learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=2, verbose=1, factor=0.35, min_lr=0.00001)
        early_stop = EarlyStopping(monitor='val_acc', patience=5, verbose = 1)
        # Common params 
        epochs = 15
        batch_size = 128
        # Create and compile the model
        model = create(act, n)
        # Compile the model
        model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
        # Train the model
        history = model.fit(X_train,Y_train, epochs = epochs, validation_data = (X_val,Y_val),
                            verbose = 1 , callbacks=[learning_rate_reduction, early_stop])
        
        # Record accuracy of each model and save it
        logger.append(model.evaluate(X_test, Y_test)[1])
        logs.info("Accuracy "+str(i)+": "+str(logger[-1]))
        # Calculate probabilities of test data and sum them toghether
        ensembler += model.predict_proba(X_test)
        # Clear session (GPU MEMORY)
        K.get_session().close()
        K.set_session(K.tf.Session())
        del model, history, learning_rate_reduction, early_stop, opt
     
    # Calculate the median accuracy
    ensembled = accuracy(ensembler, Y_test)
    print(ensembled)
    # Save the ensembled accuracy and the three models accuracy
    record_e_swish_2.append([n, ensembled])
    logs_e_swish_2.append(logger)
    del ensembler, ensembled
    logs.info("Ensembled accuracy: "+str(record_e_swish_2[-1]))
    logs.info("Logs: +"+str(logs_e_swish_2[-1]))
    
logs.info("\n \n \n")
logs.info("Logs e_swish_2: "+str(logs_e_swish_2))
logs.info("Record e_swish_2: "+str(record_e_swish_2))
    
print("\n \n \n")
print("Logs e_swish_2: ", logs_e_swish_2)
print("Record e_swish_2: ", record_e_swish_2)
    
plot([[x[1] for x in logs_relu], [x[1] for x in logs_e_swish_2]])


 
 Starting round with 23 layers
Train on 54000 samples, validate on 6000 samples
Epoch 1/15
54000/54000 [==============================] - 93s 2ms/step - loss: 1.2903 - acc: 0.5598 - val_loss: 0.5266 - val_acc: 0.8643
Epoch 2/15
54000/54000 [==============================] - 77s 1ms/step - loss: 0.7332 - acc: 0.8116 - val_loss: 0.3981 - val_acc: 0.9258
Epoch 3/15
54000/54000 [==============================] - 77s 1ms/step - loss: 0.5698 - acc: 0.8682 - val_loss: 0.3337 - val_acc: 0.9413
Epoch 4/15
54000/54000 [==============================] - 77s 1ms/step - loss: 0.5351 - acc: 0.8743 - val_loss: 0.2820 - val_acc: 0.9455
Epoch 5/15
54000/54000 [==============================] - 77s 1ms/step - loss: 0.4442 - acc: 0.9004 - val_loss: 0.2380 - val_acc: 0.9552
Epoch 6/15
54000/54000 [==============================] - 77s 1ms/step - loss: 0.3893 - acc: 0.9156 - val_loss: 0.1961 - val_acc: 0.9615
Epoch 7/15
54000/54000 [==============================] - 77s 1ms/step - loss: 0.3781 - acc: 0.

Epoch 4/15
54000/54000 [==============================] - 98s 2ms/step - loss: 0.9554 - acc: 0.7049 - val_loss: 0.6452 - val_acc: 0.7932
Epoch 5/15
54000/54000 [==============================] - 98s 2ms/step - loss: 0.8421 - acc: 0.7467 - val_loss: 0.5023 - val_acc: 0.8502
Epoch 6/15
54000/54000 [==============================] - 98s 2ms/step - loss: 0.7537 - acc: 0.7876 - val_loss: 0.4633 - val_acc: 0.8952
Epoch 7/15
54000/54000 [==============================] - 98s 2ms/step - loss: 0.7391 - acc: 0.8039 - val_loss: 0.4733 - val_acc: 0.8892
Epoch 8/15
54000/54000 [==============================] - 98s 2ms/step - loss: 0.6776 - acc: 0.8312 - val_loss: 0.3940 - val_acc: 0.9142
Epoch 9/15
54000/54000 [==============================] - 98s 2ms/step - loss: 0.6372 - acc: 0.8457 - val_loss: 0.3853 - val_acc: 0.9140
Epoch 10/15
54000/54000 [==============================] - 98s 2ms/step - loss: 0.6439 - acc: 0.8404 - val_loss: 0.3843 - val_acc: 0.9120
Epoch 11/15
54000/54000 [===============

54000/54000 [==============================] - 120s 2ms/step - loss: 0.8543 - acc: 0.7458 - val_loss: 0.4832 - val_acc: 0.8860
Epoch 14/15
54000/54000 [==============================] - 120s 2ms/step - loss: 0.8199 - acc: 0.7600 - val_loss: 0.4405 - val_acc: 0.8902
Epoch 15/15
10000/10000 [==============================] - 5s 532us/step
0.9047

 
 Starting round with 38 layers
Train on 54000 samples, validate on 6000 samples
Epoch 1/15
54000/54000 [==============================] - 185s 3ms/step - loss: 2.1707 - acc: 0.2288 - val_loss: 1.6790 - val_acc: 0.3395
Epoch 2/15
54000/54000 [==============================] - 130s 2ms/step - loss: 1.7943 - acc: 0.3041 - val_loss: 1.5162 - val_acc: 0.3598
Epoch 3/15
54000/54000 [==============================] - 130s 2ms/step - loss: 1.6937 - acc: 0.3266 - val_loss: 1.4352 - val_acc: 0.3845
Epoch 4/15
54000/54000 [==============================] - 133s 2ms/step - loss: 1.6118 - acc: 0.3547 - val_loss: 1.3481 - val_acc: 0.4338
Epoch 5/15
54000/54

In [12]:
act = e_swish_2
# logs.info("\n \n \n"+"\n \n \n")
# logs.info("ESWISH 41-42")

logs_e_swish_2 = []
record_e_swish_2 = []
for n in range(41,45,3):
    ensembler = 0
    logger = [n]
    logs.info("\n \n Starting round with {0} layers".format(n))
    print("\n \n Starting round with {0} layers".format(n))
    for i in range(3):
        # Garbage collector
        gc.collect()
        # Set optimizer
        opt = SGD(lr=0.01, momentum=0.9)
        # Set callbacks (learning rate reducer and early stopping)
        learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=2, verbose=1, factor=0.35, min_lr=0.00001)
        early_stop = EarlyStopping(monitor='val_acc', patience=5, verbose = 1)
        # Common params 
        epochs = 15
        batch_size = 128
        # Create and compile the model
        model = create(act, n)
        # Compile the model
        model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
        # Train the model
        history = model.fit(X_train,Y_train, epochs = epochs, validation_data = (X_val,Y_val),
                            verbose = 1 , callbacks=[learning_rate_reduction, early_stop])
        
        # Record accuracy of each model and save it
        logger.append(model.evaluate(X_test, Y_test)[1])
        logs.info("Accuracy "+str(i)+": "+str(logger[-1]))
        # Calculate probabilities of test data and sum them toghether
        ensembler += model.predict_proba(X_test)
        # Clear session (GPU MEMORY)
        K.get_session().close()
        K.set_session(K.tf.Session())
        del model, history, learning_rate_reduction, early_stop, opt
     
    # Calculate the median accuracy
    ensembled = accuracy(ensembler, Y_test)
    print(ensembled)
    # Save the ensembled accuracy and the three models accuracy
    record_e_swish_2.append([n, ensembled])
    logs_e_swish_2.append(logger)
    del ensembler, ensembled
    logs.info("Ensembled accuracy: "+str(record_e_swish_2[-1]))
    logs.info("Logs: +"+str(logs_e_swish_2[-1]))
    
# logs.info("\n \n \n")
# logs.info("Logs e_swish_2: "+str(logs_e_swish_2))
# logs.info("Record e_swish_2: "+str(record_e_swish_2))
    
# print("\n \n \n")
# print("Logs e_swish_2: ", logs_e_swish_2)
# print("Record e_swish_2: ", record_e_swish_2)


 
 Starting round with 41 layers
Train on 54000 samples, validate on 6000 samples
Epoch 1/15
54000/54000 [==============================] - 140s 3ms/step - loss: 2.3038 - acc: 0.1406 - val_loss: 2.0706 - val_acc: 0.1923
Epoch 2/15
54000/54000 [==============================] - 134s 2ms/step - loss: 2.0379 - acc: 0.2082 - val_loss: 1.6829 - val_acc: 0.3590
Epoch 3/15
54000/54000 [==============================] - 134s 2ms/step - loss: 1.8362 - acc: 0.2873 - val_loss: 1.5304 - val_acc: 0.3847
Epoch 4/15
54000/54000 [==============================] - 133s 2ms/step - loss: 1.7062 - acc: 0.3286 - val_loss: 1.3722 - val_acc: 0.4520
Epoch 5/15
54000/54000 [==============================] - 132s 2ms/step - loss: 1.6300 - acc: 0.3511 - val_loss: 1.3249 - val_acc: 0.4560
Epoch 6/15
54000/54000 [==============================] - 132s 2ms/step - loss: 1.5921 - acc: 0.3683 - val_loss: 1.2773 - val_acc: 0.4715
Epoch 7/15
54000/54000 [==============================] - 132s 2ms/step - loss: 1.5210 - 

Epoch 12/15
54000/54000 [==============================] - 144s 3ms/step - loss: 1.7652 - acc: 0.2927 - val_loss: 1.6418 - val_acc: 0.3443
Epoch 13/15
54000/54000 [==============================] - 143s 3ms/step - loss: 1.7439 - acc: 0.2987 - val_loss: 1.6365 - val_acc: 0.3762
Epoch 14/15
54000/54000 [==============================] - 145s 3ms/step - loss: 1.7236 - acc: 0.3065 - val_loss: 1.8872 - val_acc: 0.3463
Epoch 15/15
10000/10000 [==============================] - 5s 546us/step
Train on 54000 samples, validate on 6000 samples
Epoch 1/15
54000/54000 [==============================] - 159s 3ms/step - loss: 2.4932 - acc: 0.1228 - val_loss: 2.2272 - val_acc: 0.0997
Epoch 2/15
54000/54000 [==============================] - 143s 3ms/step - loss: 2.2148 - acc: 0.1710 - val_loss: 2.1071 - val_acc: 0.1945
Epoch 3/15
54000/54000 [==============================] - 144s 3ms/step - loss: 2.1358 - acc: 0.1880 - val_loss: 2.0248 - val_acc: 0.2258
Epoch 4/15
54000/54000 [=======================

In [13]:
logs.info("\n \n \n")
logs.info("Logs e_swish_2: "+str(logs_e_swish_2))
logs.info("Record e_swish_2: "+str(record_e_swish_2))
    
print("\n \n \n")
print("Logs e_swish_2: ", logs_e_swish_2)
print("Record e_swish_2: ", record_e_swish_2)


 
 

Logs e_swish_2:  [[41, 0.6018, 0.55249999999999999, 0.48520000000000002], [44, 0.35160000000000002, 0.51910000000000001, 0.3891]]
Record e_swish_2:  [[41, 0.679], [44, 0.6189]]


In [12]:
act = swish
logs.info("\n \n \n"+"\n \n \n")
logs.info("SWISH 23-42")

logs_swish = []
record_swish = []
for n in range(23,45,3):
    ensembler = 0
    logger = [n]
    logs.info("\n \n Starting round with {0} layers".format(n))
    print("\n \n Starting round with {0} layers".format(n))
    for i in range(3):
        # Garbage collector
        gc.collect()
        # Set optimizer
        opt = SGD(lr=0.01, momentum=0.9)
        # Set callbacks (learning rate reducer and early stopping)
        learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=2, verbose=1, factor=0.35, min_lr=0.00001)
        early_stop = EarlyStopping(monitor='val_acc', patience=5, verbose = 1)
        # Common params 
        epochs = 15
        batch_size = 128
        # Create and compile the model
        model = create(act, n)
        # Compile the model
        model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
        # Train the model
        history = model.fit(X_train,Y_train, epochs = epochs, validation_data = (X_val,Y_val),
                            verbose = 1 , callbacks=[learning_rate_reduction, early_stop])
        
        # Record accuracy of each model and save it
        logger.append(model.evaluate(X_test, Y_test)[1])
        logs.info("Accuracy "+str(i)+": "+str(logger[-1]))
        # Calculate probabilities of test data and sum them toghether
        ensembler += model.predict_proba(X_test)
        # Clear session (GPU MEMORY)
        K.get_session().close()
        K.set_session(K.tf.Session())
        del model, history, learning_rate_reduction, early_stop, opt
     
    # Calculate the median accuracy
    ensembled = accuracy(ensembler, Y_test)
    print(ensembled)
    # Save the ensembled accuracy and the three models accuracy
    record_swish.append([n, ensembled])
    logs_swish.append(logger)
    del ensembler, ensembled
    logs.info("Ensembled accuracy: "+str(record_swish[-1]))
    logs.info("Logs: +"+str(logs_swish[-1]))
    
logs.info("\n \n \n")
logs.info("Logs swish: "+str(logs_swish))
logs.info("Record swish: "+str(record_swish))
    
print("\n \n \n")
print("Logs swish: ", logs_swish)
print("Record swish: ", record_swish)


 
 Starting round with 23 layers
Train on 54000 samples, validate on 6000 samples
Epoch 1/15
54000/54000 [==============================] - 75s 1ms/step - loss: 1.0462 - acc: 0.6580 - val_loss: 0.3307 - val_acc: 0.9118
Epoch 2/15
54000/54000 [==============================] - 71s 1ms/step - loss: 0.5330 - acc: 0.8714 - val_loss: 0.2276 - val_acc: 0.9472
Epoch 3/15
54000/54000 [==============================] - 71s 1ms/step - loss: 0.4287 - acc: 0.9022 - val_loss: 0.1684 - val_acc: 0.9613
Epoch 4/15
54000/54000 [==============================] - 71s 1ms/step - loss: 0.3613 - acc: 0.9221 - val_loss: 0.1670 - val_acc: 0.9608
Epoch 5/15
54000/54000 [==============================] - 71s 1ms/step - loss: 0.3216 - acc: 0.9314 - val_loss: 0.1519 - val_acc: 0.9670
Epoch 6/15
54000/54000 [==============================] - 71s 1ms/step - loss: 0.2884 - acc: 0.9379 - val_loss: 0.1291 - val_acc: 0.9715
Epoch 7/15
54000/54000 [==============================] - 71s 1ms/step - loss: 0.2596 - acc: 0.

54000/54000 [==============================] - 89s 2ms/step - loss: 0.6429 - acc: 0.8598 - val_loss: 0.3507 - val_acc: 0.9335
Epoch 6/15
54000/54000 [==============================] - 89s 2ms/step - loss: 0.5708 - acc: 0.8796 - val_loss: 0.3894 - val_acc: 0.9255
Epoch 7/15
54000/54000 [==============================] - 89s 2ms/step - loss: 0.5816 - acc: 0.8753 - val_loss: 0.3646 - val_acc: 0.9255
Epoch 8/15
53984/54000 [============================>.] - ETA: 0s - loss: 0.5460 - acc: 0.8831
Epoch 00008: reducing learning rate to 0.0034999999217689036.
54000/54000 [==============================] - 90s 2ms/step - loss: 0.5461 - acc: 0.8830 - val_loss: 0.4538 - val_acc: 0.8997
Epoch 9/15
54000/54000 [==============================] - 89s 2ms/step - loss: 0.4725 - acc: 0.9004 - val_loss: 0.2715 - val_acc: 0.9467
Epoch 10/15
54000/54000 [==============================] - 89s 2ms/step - loss: 0.3936 - acc: 0.9213 - val_loss: 0.2557 - val_acc: 0.9520
Epoch 11/15
54000/54000 [=================

53984/54000 [============================>.] - ETA: 0s - loss: 1.4624 - acc: 0.4843
Epoch 00006: reducing learning rate to 0.0034999999217689036.
54000/54000 [==============================] - 121s 2ms/step - loss: 1.4623 - acc: 0.4843 - val_loss: 1.2412 - val_acc: 0.5240
Epoch 7/15
54000/54000 [==============================] - 119s 2ms/step - loss: 1.3307 - acc: 0.5085 - val_loss: 1.0924 - val_acc: 0.5707
Epoch 8/15
53984/54000 [============================>.] - ETA: 0s - loss: 1.2730 - acc: 0.5207
Epoch 00008: reducing learning rate to 0.0012249999563209713.
54000/54000 [==============================] - 118s 2ms/step - loss: 1.2730 - acc: 0.5207 - val_loss: 1.1310 - val_acc: 0.5563
Epoch 00008: early stopping
10000/10000 [==============================] - 5s 483us/step
Train on 54000 samples, validate on 6000 samples
Epoch 1/15
54000/54000 [==============================] - 155s 3ms/step - loss: 1.6491 - acc: 0.3853 - val_loss: 1.0922 - val_acc: 0.5350
Epoch 2/15
54000/54000 [=====

Epoch 00007: early stopping
10000/10000 [==============================] - 6s 553us/step
0.6553

 
 Starting round with 44 layers
Train on 54000 samples, validate on 6000 samples
Epoch 1/15
54000/54000 [==============================] - 189s 3ms/step - loss: 1.7660 - acc: 0.3470 - val_loss: 1.1964 - val_acc: 0.4793
Epoch 2/15
54000/54000 [==============================] - 138s 3ms/step - loss: 1.4717 - acc: 0.4286 - val_loss: 1.4418 - val_acc: 0.3628
Epoch 3/15
54000/54000 [==============================] - 139s 3ms/step - loss: 1.5745 - acc: 0.3834 - val_loss: 1.3689 - val_acc: 0.4892
Epoch 4/15
54000/54000 [==============================] - 138s 3ms/step - loss: 1.4541 - acc: 0.4387 - val_loss: 1.3224 - val_acc: 0.4720
Epoch 5/15
54000/54000 [==============================] - 138s 3ms/step - loss: 1.6842 - acc: 0.3848 - val_loss: 1.7230 - val_acc: 0.3873
Epoch 6/15
53984/54000 [============================>.] - ETA: 0s - loss: 1.6507 - acc: 0.3629
Epoch 00006: reducing learning rate 